In [1]:
import uproot
import ROOT
import numpy as np
import pandas as pd
from collections import OrderedDict
from coffea import hist

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['grid.linestyle'] = ':'

Welcome to JupyROOT 6.12/04


In [2]:
import sys
print(sys.version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]


In [3]:
import matplotlib as mpl
mpl.style.use('seaborn-bright')

In [4]:
web_dir = '/publicweb/m/mreid/iDM_AN_Plots/RecoStudies'

# Define functions

In [5]:
def extract_teffi(eff):
    _ = ROOT.TCanvas()
    eff.Draw()
    ROOT.gPad.Update()
    
    tot = eff.GetTotalHistogram()
    nBins = tot.GetNbinsX()
    xx = []
    yy = []
    yl = []
    yu = []
    for i in range(1, nBins+1):
        if eff.GetEfficiency(eff.GetGlobalBin(i)) == 0 and eff.GetEfficiencyErrorUp(eff.GetGlobalBin(i)) == 1.: continue
        if eff.GetEfficiency(eff.GetGlobalBin(i)) == 1. and eff.GetEfficiencyErrorLow(eff.GetGlobalBin(i)) == 1.: continue
        
        xx.append(tot.GetXaxis().GetBinCenter(i))
        yy.append(eff.GetEfficiency(eff.GetGlobalBin(i)))
        yl.append(eff.GetEfficiencyErrorLow(eff.GetGlobalBin(i)))
        yu.append(eff.GetEfficiencyErrorUp(eff.GetGlobalBin(i)))
                  
    
    return {'x': np.array(xx),
            'y': np.array(yy),
            'yerr': np.array([yl, yu])}

In [6]:
import scipy.special
from scipy.optimize import curve_fit, fsolve
from scipy import odr

def fsigmoid(z, p0, p1):
    return 1.0 / (1.0 + np.exp(-p0*(z-p1)))
def ferf_cf(z, p0, p1, p2, p3):
    return p2 + p3*scipy.special.erf((z-p0)/p1)
def f98percentile_cf(z, p):
    return ferf_cf(z, *p) - (p[2]+p[3])*0.98
def ferf_odr(p, z):
    return p[2] + p[3]*scipy.special.erf((z-p[0])/p[1])
def f98percentile_odr(z, p):
    return ferf_odr(p, z) - (p[2]+p[3])*0.98

def make_plot(ax, sample, objs, variable, plot_props, event_selection='df'):
    h0 = ROOT.TH1F(f'h0_{sample}', '', len(plot_props['axis'])-1, plot_props['axis'])
    h1 = ROOT.TH1F(f'h1_{sample}', '', len(plot_props['axis'])-1, plot_props['axis'])
    for l,x in objs[event_selection][variable].iteritems():
        h0.Fill(x)
    for l,x in objs[event_selection][objs[event_selection]['nMatched']>0][variable].iteritems():
        h1.Fill(x)
    eff = ROOT.TEfficiency(h1, h0)
    data = extract_teffi(eff)
    #print(data)
    xerr = np.insert(np.diff(data['x'])/2, 0, (data['x'][1]-data['x'][0])/2)
    ax.errorbar(x=data['x'], y=data['y'], xerr=xerr, yerr=data['yerr'], 
                label=f'{props["label"]}', markersize=3, alpha=0.3, fmt='o', c=objs['color'])
    #ax.plot(x=data['x'],y=data['y'])#, label=f'{props["label"]}', markersize=3, alpha=0.3, fmt='o', c=objs['color'])
    selection_string = 'dsa' if event_selection == 'dsa_df' else 'gbm'
    units = '[GeV]' if 'Pt'in variable else '[mm]'
    ax.set_title(f'{selection_string} muon reconstruction as a function of {variable} (over Gen Muons)')
    ax.set_xlabel(f'{variable} {units}')
    ax.set_ylabel('Muon Reconstruction efficiency')
    ax.set_ylim((0,1))
    ax.legend()
    
    return data

def make_plot_fit_cf(ax, sample, objs, variable, plot_props, event_selection='df'):
    data = make_plot(ax, sample, objs, variable, plot_props, event_selection)

    minbin = 0 if data['y'][0] < 0.1 else 1
    maxbin = -1
    popt, pcov = curve_fit(ferf_cf, data['x'][minbin:maxbin], data['y'][minbin:maxbin], sigma=data['yerr'][1][minbin:maxbin],
                           bounds=([50,0,-10,-10],[250,100,10,10]))
    print(fr"[{props['label']}]")
    print('fit parameter 1-sigma error')
    print('———————————–')
    for i in range(len(popt)):
        print(f'{popt[i]:.2f} +- {pcov[i,i]:.2f}')
    print('')
    
    percentile98 = fsolve(f98percentile_cf, 200, popt)
    fit_range = np.linspace(data['x'][minbin], data['x'][maxbin], 100)
    ax.plot(fit_range, ferf_cf(fit_range, *popt), c=objs['color'],
            label=f'Fit: pT @ 98% = {percentile98[0]:.0f} GeV')
    ax.legend()
    
def make_plot_fit_odr(ax, sample, objs, variable, plot_props, event_selection='df'):
    data = make_plot(ax, sample, objs, variable, plot_props, event_selection)

    minbin = 0 if data['y'][0] < 0.1 else 1
    maxbin = -1
    
    model = odr.Model(ferf_odr)
    xerror = np.repeat(data['x'][1]-data['x'][0], len(data['x'])-minbin)
    real_data = odr.RealData(data['x'][minbin:maxbin], data['y'][minbin:maxbin], sx=xerror, sy=data['yerr'][0][minbin:maxbin])
    odr_obj = odr.ODR(real_data, model, beta0=[132, 36, 0.5, 0.5])
    out = odr_obj.run()
    
    popt = out.beta
    perr = out.sd_beta
    print(fr"[{props['label']}]")
    print('fit parameter 1-sigma error')
    print('———————————–')
    for i in range(len(popt)):
        print(f'{popt[i]:.2f} +- {perr[i]:.2f}')
    print('')
    
    percentile98 = fsolve(f98percentile_odr, 200, popt)
    fit_range = np.linspace(data['x'][minbin], data['x'][maxbin], 100)
    ax.plot(fit_range, ferf_odr(popt, fit_range), c=objs['color'],
            label=f'Fit: pT @ 98% = {percentile98[0]:.0f} GeV')
    
    ax.legend()

# Load data

In [7]:
redirector = 'root://cmseos.fnal.gov/'
basedir = '/store/group/lpcmetx/iDM/Ntuples/2018/signal/reco_effi'
samples = {
    '5p25-1': {
        'filename': redirector + basedir + '/Mchi-5p25_dMchi-0p5_ctau-1/iDMAnalysis_Mchi-5p25_dMchi-0p5_ctau-1.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.1$, ctau=1',
        'mass': '5p25',
        'ctau': '1',
        'color': 'C0'
    },
    '5p25-10': {
        'filename': redirector + basedir + '/Mchi-5p25_dMchi-0p5_ctau-10/iDMAnalysis_Mchi-5p25_dMchi-0p5_ctau-10.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.1$, ctau=10',
        'mass': '5p25',
        'ctau': '10',
        'color': 'C1'
    },
    '5p25-100': {
        'filename': redirector + basedir + '/Mchi-5p25_dMchi-0p5_ctau-100/iDMAnalysis_Mchi-5p25_dMchi-0p5_ctau-100.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.1$, ctau=100',
        'mass': '5p25',
        'ctau': '100',
        'color': 'C2'
    },
    '5p25-1000': {
        'filename': redirector + basedir + '/Mchi-5p25_dMchi-0p5_ctau-1000/iDMAnalysis_Mchi-5p25_dMchi-0p5_ctau-1000.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.1$, ctau=1000',
        'mass': '5p25',
        'ctau': '1000',
        'color': 'C3'
    },
    '52p5-1': {
        'filename': redirector + basedir + '/Mchi-52p5_dMchi-5p0_ctau-1/iDMAnalysis_Mchi-52p5_dMchi-5p0_ctau-1.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.1$, ctau=1',
        'mass': '52p5',
        'ctau': '1',
        'color': 'C1'
    },
    '52p5-10': {
        'filename': redirector + basedir + '/Mchi-52p5_dMchi-5p0_ctau-10/iDMAnalysis_Mchi-52p5_dMchi-5p0_ctau-10.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.1$, ctau=10',
        'mass': '52p5',
        'ctau': '10',
        'color': 'C2'
    },
    '52p5-100': {
        'filename': redirector + basedir + '/Mchi-52p5_dMchi-5p0_ctau-100/iDMAnalysis_Mchi-52p5_dMchi-5p0_ctau-100.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.1$, ctau=100',
        'mass': '52p5',
        'ctau': '100',
        'color': 'C3'
    },
    '52p5-1000': {
        'filename': redirector + basedir + '/Mchi-52p5_dMchi-5p0_ctau-1000/iDMAnalysis_Mchi-52p5_dMchi-5p0_ctau-1000.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.1$, ctau=1000',
        'mass': '52p5',
        'ctau': '1000',
        'color': 'C0'
    },
    '6p0-1': {
        'filename': redirector + basedir + '/Mchi-6p0_dMchi-2p0_ctau-1/iDMAnalysis_Mchi-6p0_dMchi-2p0_ctau-1.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.4$, ctau=1',
        'mass': '6p0',
        'ctau': '1',
        'color': 'C2'
    },
    '6p0-10': {
        'filename': redirector + basedir + '/Mchi-6p0_dMchi-2p0_ctau-10/iDMAnalysis_Mchi-6p0_dMchi-2p0_ctau-10.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.4$, ctau=10',
        'mass': '6p0',
        'ctau': '10',
        'color': 'C3'
    },
    '6p0100': {
        'filename': redirector + basedir + '/Mchi-6p0_dMchi-2p0_ctau-100/iDMAnalysis_Mchi-6p0_dMchi-2p0_ctau-100.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.4$, ctau=100',
        'mass': '6p0',
        'ctau': '100',
        'color': 'C0'
    },
    '6p0-1000': {
        'filename': redirector + basedir + '/Mchi-6p0_dMchi-2p0_ctau-1000/iDMAnalysis_Mchi-6p0_dMchi-2p0_ctau-1000.root',
        'label': r'm1 = 5 GeV, $\Delta = 0.4$, ctau=1000',
        'mass': '6p0',
        'ctau': '1000',
        'color': 'C1'
    },
    '60p0-1': {
        'filename': redirector + basedir + '/Mchi-60p0_dMchi-20p0_ctau-1/iDMAnalysis_Mchi-60p0_dMchi-20p0_ctau-1.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.4$, ctau=1',
        'mass': '60p0',
        'ctau': '1',
        'color': 'C3'
    },
    '60p0-10': {
        'filename': redirector + basedir + '/Mchi-60p0_dMchi-20p0_ctau-10/iDMAnalysis_Mchi-60p0_dMchi-20p0_ctau-10.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.4$, ctau=10',
        'mass': '60p0',
        'ctau': '10',
        'color': 'C0'
    },
    '60p0-100': {
        'filename': redirector + basedir + '/Mchi-60p0_dMchi-20p0_ctau-100/iDMAnalysis_Mchi-60p0_dMchi-20p0_ctau-100.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.4$, ctau=100',
        'mass': '60p0',
        'ctau': '100',
        'color': 'C1'
    },
    '60p0-1000': {
        'filename': redirector + basedir + '/Mchi-60p0_dMchi-20p0_ctau-1000/iDMAnalysis_Mchi-60p0_dMchi-20p0_ctau-1000.root',
        'label': r'm1 = 50 GeV, $\Delta = 0.4$, ctau=1000',
        'mass': '60p0',
        'ctau': '1000',
        'color': 'C2'
    },

}

In [ ]:
for sample, objs in samples.items():
    file = uproot.open(objs['filename'])
    objs['dsa_tree'] = file['RECO_dsa/recoEffiForMuTrack']
    objs['gbm_tree'] = file['RECO_gbm/recoEffiForMuTrack']
    objs['dsa_df_gen'] = objs['dsa_tree'].pandas.df(['genPt','genEta','genPhi','genVxy','genVz','genDr'])
    objs['dsa_df_reco'] = objs['dsa_tree'].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz','deltaR'])
    objs['dsa_df_jetmet'] = objs['dsa_tree'].pandas.df(['nMatched','genJetPt','genJetEta','genJetPhi','genLeadMetPt','genLeadMetPhi','recoPFMetPt'])
    objs['gbm_df_gen'] = objs['gbm_tree'].pandas.df(['genPt','genEta','genPhi','genVxy','genVz','genDr'])
    objs['gbm_df_reco'] = objs['gbm_tree'].pandas.df(['recoPt','recoEta','recoPhi','recoDxy','recoDz','deltaR'])
    objs['gbm_df_jetmet'] = objs['gbm_tree'].pandas.df(['nMatched','genJetPt','genJetEta','genJetPhi','genLeadMetPt','genLeadMetPhi','recoPFMetPt'])
    objs['dsa_df'] = objs['dsa_df_gen'].join(objs['dsa_df_reco']).join(objs['dsa_df_jetmet'])
    objs['gbm_df'] = objs['gbm_df_gen'].join(objs['gbm_df_reco']).join(objs['gbm_df_jetmet'])
    #print(objs['dsa_df'][['genPt','nMatched']])
#    print(objs['dsa_df'][objs['dsa_df']['nMatched']>0.1][['genPt','nMatched']])

In [ ]:
#for l,x in objs['dsa_df'][objs['dsa_df']['nMatched']>0]['nMatched'].iteritems():
#    print(x)

In [ ]:
eff_plots = OrderedDict({
    'genPt': {
        'axis': np.array(np.append(np.arange(0,22,3), [30]), dtype='float64'),
        #'axis': np.array(np.arange(0,30,30), dtype='float64'),
        'range': [0, 30],
        'label': 'Leading gen muon pT [GeV]'
    },
    'genEta': {
        'axis': np.array(np.linspace(-2.4,2.4,6), dtype='float64'),
        'range': [-2.5, 2.5],
        'label': 'Leading gen muon eta'
    },
    'genPhi': {
        'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
        'range': [-3.2, 3.2],
        'label': 'Leading gen muon phi'
    },
    'genVxy': {
        'axis': np.array(np.linspace(0,1000,20), dtype='float64'),
        #'axis': np.array(np.append(np.arange(0,100,20), [30]), dtype='float64'),
        'range': [0, 1000],
        'label': 'gen muon Vxy'
    },
    'genVz': {
        'axis': np.array(np.linspace(-50,50,20), dtype='float64'),
        'range': [-500, 500],
        'label': 'gen muon Vz'
    },
    'recoPt': {
        'axis': np.array(np.arange(30,300,30), dtype='float64'),
        'range': [0, 30],
        'label': 'Leading reco muon pT [GeV]'
    },
    'recoEta': {
        'axis': np.array(np.linspace(-2.4,2.4,6), dtype='float64'),
        'range': [-2.5, 2.5],
        'label': 'Leading reco muon eta'
    },
    'recoPhi': {
        'axis': np.array(np.linspace(-3.1415,3.1415,6), dtype='float64'),
        'range': [-3.2, 3.2],
        'label': 'Leading reco muon phi'
    },
    'genLeadMetPt': {
        'axis': np.array(np.arange(30,300,30), dtype='float64'),
        'range': [0, 500],
        'label': 'MET pT [GeV]'
    },
})

# DSA Muons

In [ ]:
#sample = '52p5-100'
for sample in samples:
    fig, axes = plt.subplots(3,3, figsize=(14,10))
    fig.suptitle(f'Matched vs All Muons ({samples[sample]["label"]})')
    plt.tight_layout(h_pad=2.0)
    fig.subplots_adjust(top=0.90)
    binn= 2000 if samples[sample]['ctau']=='1000' else 5*int(samples[sample]['ctau'])
    eff_plots['genVxy']['range']= [0,binn]
    for index, (column, props) in enumerate(eff_plots.items()):
        ax = axes[index//3, index%3]
    
        bin_edges = props['axis']
        kwargs = {'range': props['range'], 'bins':10, 'density': True, 'histtype':'step','log':False}
    
        df = samples[sample]['dsa_df']
        ax.hist(df[column].groupby('entry').nth(0), **kwargs, 
                label='All muons');
        ax.hist(df.query('nMatched>0')[column].groupby('entry').nth(0), **kwargs, 
                label='>0 Matched muons');
        ax.hist(df.query('nMatched==1')[column].groupby('entry').nth(0), **kwargs, 
                label='1 Matched muon');
        ax.hist(df.query('nMatched==2')[column].groupby('entry').nth(0), **kwargs, 
                label='2 Matched muons');
        ax.set_xlabel(props['label'])
        ax.set_ylabel('A. U.')

    ax.legend(loc=(1.2 ,0.4))
    fig.savefig(f'{web_dir}/dsamatched_vs_allMu_{sample}', bbox_inches='tight')

In [ ]:
#sample = '52p5-100'
for sample in samples:
    fig, axes = plt.subplots(3,3, figsize=(14,10))
    fig.suptitle(f'Matched vs All Muons ({samples[sample]["label"]})')
    plt.tight_layout(h_pad=2.0)
    fig.subplots_adjust(top=0.90)
    binn= 2000 if samples[sample]['ctau']=='1000' else 5*int(samples[sample]['ctau'])
    eff_plots['genVxy']['range']= [0,binn]

    for index, (column, props) in enumerate(eff_plots.items()):
        ax = axes[index//3, index%3]
    
        bin_edges = props['axis']
        kwargs = {'range': props['range'], 'bins':10, 'density': True, 'histtype':'step','log':True}
    
        df = samples[sample]['dsa_df']
        ax.hist(df[column].groupby('entry').nth(0), **kwargs, 
            label='All muons');
        ax.hist(df.query('nMatched>0')[column].groupby('entry').nth(0), **kwargs, 
            label='>0 Matched muons');
        ax.hist(df.query('nMatched==1')[column].groupby('entry').nth(0), **kwargs, 
            label='1 Matched muon');
        ax.hist(df.query('nMatched==2')[column].groupby('entry').nth(0), **kwargs, 
            label='2 Matched muons');
        ax.set_xlabel(props['label'])
        ax.set_ylabel('A. U.')

    ax.legend(loc=(1.2 ,0.4))
    fig.savefig(f'{web_dir}/dsamatched_vs_allMu_log_{sample}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for mass in mchis:
    fig, ax = plt.subplots(figsize=(8,6))
    for ctau in ctaus:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genPt', eff_plots['genPt'], event_selection='dsa_df')
    fig.savefig(f'{web_dir}/dsaReco_vs_genPt_mass_{mass}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for ctau in ctaus:
    fig, ax = plt.subplots(figsize=(8,6))
    for mass in mchis:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genPt', eff_plots['genPt'], event_selection='dsa_df')
    fig.savefig(f'{web_dir}/dsaReco_vs_genPt_ctau_{ctau}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for mass in mchis:
    fig, ax = plt.subplots(figsize=(8,6))
    eff_plots['genVxy']['axis']= np.array(np.linspace(0,50,20), dtype='float64')
    for ctau in ctaus:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genVxy', eff_plots['genVxy'], event_selection='dsa_df')
    fig.savefig(f'{web_dir}/dsaReco_vs_genVxy_mass_{mass}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for ctau in ctaus:
    fig, ax = plt.subplots(figsize=(8,6))
    binn= 200 if ctau=='1000' else 30
    eff_plots['genVxy']['axis']= np.array(np.linspace(0,5*int(ctau),binn), dtype='float64')
    for mass in mchis:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genVxy', eff_plots['genVxy'], event_selection='dsa_df')
    fig.savefig(f'{web_dir}/dsaReco_vs_genVxy_ctau_{ctau}.png', bbox_inches='tight')

# gbm muons

In [ ]:
#sample = '52p5-100'
for sample in samples:
    fig, axes = plt.subplots(3,3, figsize=(14,10))
    fig.suptitle(f'Matched vs All Muons ({samples[sample]["label"]})')
    plt.tight_layout(h_pad=2.0)
    fig.subplots_adjust(top=0.90)
    binn= 2000 if samples[sample]['ctau']=='1000' else 5*int(samples[sample]['ctau'])
    eff_plots['genVxy']['range']= [0,binn]
    for index, (column, props) in enumerate(eff_plots.items()):
        ax = axes[index//3, index%3]
        
        bin_edges = props['axis']
        kwargs = {'range': props['range'], 'bins':10, 'density': True, 'histtype':'step','log':False}
    
        df = samples[sample]['gbm_df']
        ax.hist(df[column].groupby('entry').nth(0), **kwargs, 
            label='All muons');
        ax.hist(df.query('nMatched>0')[column].groupby('entry').nth(0), **kwargs, 
            label='>0 Matched muons');
        ax.hist(df.query('nMatched==1')[column].groupby('entry').nth(0), **kwargs, 
            label='1 Matched muon');
        ax.hist(df.query('nMatched==2')[column].groupby('entry').nth(0), **kwargs, 
            label='2 Matched muons');
        ax.set_xlabel(props['label'])
        ax.set_ylabel('A. U.')

    ax.legend(loc=(1.2 ,0.4))
    fig.savefig(f'{web_dir}/gbmmatched_vs_allMu_{sample}.png', bbox_inches='tight')

In [ ]:
#sample = '52p5-100'
for sample in samples:
    fig, axes = plt.subplots(3,3, figsize=(14,10))
    fig.suptitle(f'Matched vs All Muons ({samples[sample]["label"]})')
    plt.tight_layout(h_pad=2.0)
    fig.subplots_adjust(top=0.90)
    binn= 2000 if samples[sample]['ctau']=='1000' else 5*int(samples[sample]['ctau'])
    eff_plots['genVxy']['range']= [0,binn]
    for index, (column, props) in enumerate(eff_plots.items()):
        ax = axes[index//3, index%3]
    
        bin_edges = props['axis']
        kwargs = {'range': props['range'], 'bins':10, 'density': True, 'histtype':'step','log':True}
    
        df = samples[sample]['gbm_df']
        ax.hist(df[column].groupby('entry').nth(0), **kwargs, 
            label='All muons');
        ax.hist(df.query('nMatched>0')[column].groupby('entry').nth(0), **kwargs, 
            label='>0 Matched muons');
        ax.hist(df.query('nMatched==1')[column].groupby('entry').nth(0), **kwargs, 
            label='1 Matched muon');
        ax.hist(df.query('nMatched==2')[column].groupby('entry').nth(0), **kwargs, 
            label='2 Matched muons');
        ax.set_xlabel(props['label'])
        ax.set_ylabel('A. U.')

    ax.legend(loc=(1.2 ,0.4))
    fig.savefig(f'{web_dir}/gbmmatched_vs_allMu_log_{sample}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for mass in mchis:
    fig, ax = plt.subplots(figsize=(8,6))
    for ctau in ctaus:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genPt', eff_plots['genPt'], event_selection='gbm_df')
    fig.savefig(f'{web_dir}/gbmReco_vs_genPt_mass_{mass}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for ctau in ctaus:
    fig, ax = plt.subplots(figsize=(8,6))
    for mass in mchis:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genPt', eff_plots['genPt'], event_selection='gbm_df')
    fig.savefig(f'{web_dir}/gbmReco_vs_genPt_ctau_{ctau}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for mass in mchis:
    fig, ax = plt.subplots(figsize=(8,6))
    eff_plots['genVxy']['axis']= np.array(np.linspace(0,40,20), dtype='float64')
    for ctau in ctaus:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genVxy', eff_plots['genVxy'], event_selection='gbm_df')
    fig.savefig(f'{web_dir}/gbmReco_vs_genVxy_mass_{mass}.png', bbox_inches='tight')

In [ ]:
ctaus = ['1','10','100','1000']
mchis = ['5p25','52p5','6p0','60p0']
for ctau in ctaus:
    fig, ax = plt.subplots(figsize=(8,6))
    binn= 200 if ctau=='1000' else 30
    eff_plots['genVxy']['axis']= np.array(np.linspace(0,5*int(ctau),binn), dtype='float64')
    for mass in mchis:
        for index, (sample, props) in enumerate(samples.items()):
            if (props['ctau'] != ctau) or (props['mass'] != mass):
                continue
            make_plot(ax, sample, props, 'genVxy', eff_plots['genVxy'], event_selection='gbm_df')
    fig.savefig(f'{web_dir}/gbmReco_vs_genVxy_ctau_{ctau}.png', bbox_inches='tight')